In [ ]:
import os
import getpass
from datetime import date
from google.colab import userdata

from langchain import hub
from langchain_core.tools import Tool
from langchain.tools import WikipediaQueryRun, BaseTool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI

In [ ]:
def current_day(*args, **kwargs):
    dia = date.today().strftime('%d/%m/%Y')
    return dia

date_tool = Tool(
    name="Day",
    func=current_day,
    description="Use when you need to know the current date"
)

search_tool = TavilySearchResults(max_results=2)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=3000))

wikipedia_tool = Tool(
    name="wikipedia",
    description="You must never search for multiple concepts at a single step, you need to search one at a time. When asked to compare two concepts for example, search for each one individually.",
    func=wikipedia.run
)

tools = [search_tool, date_tool]

In [ ]:
template_super_intelligence = '''<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Your name is Transcendius.
You are an artificial superintelligence. Your goal is to be a resource to access all kinds of knowledge existent.
Answer in portuguese unless being asked otherwise.
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
'''

In [ ]:
model = HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={
        "temperature": 1,
        "return_full_text": False,
        "max_new_tokens": 350,
    },
    huggingfacehub_api_token=userdata.get('HUGGINGFACEHUB_API_TOKEN')
)
agent = create_react_agent(llm=model, tools=tools, prompt=PromptTemplate.from_template(template_super_intelligence), stop_sequence=True)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handling_parsing_errors=True)